In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import wandb
import h5py
import os
import time
import json
from datetime import datetime

In [2]:
class Generator(nn.Module):
    def __init__(self, x_dim, vocab_sizes, nb_numeric, h_dims, z_dim):
        """
        Generator network for conditional GAN
        Args:
            x_dim: Dimension of output data
            vocab_sizes: List of vocabulary sizes for each categorical variable
            nb_numeric: Number of numeric covariates
            h_dims: List of hidden dimensions
            z_dim: Dimension of latent noise vector
        """
        super(Generator, self).__init__()
        
        # Embedding layers for categorical variables
        self.embeddings = nn.ModuleList([
            nn.Embedding(vocab_size, min(50, vocab_size)) 
            for vocab_size in vocab_sizes
        ])
        
        # Calculate total embedding dimension
        embedding_dim = sum(min(50, vocab_size) for vocab_size in vocab_sizes)
        
        # Input dimension is latent dim + embedding dim + numeric covariates
        input_dim = z_dim + embedding_dim + nb_numeric
        
        # Build generator network
        layers = []
        current_dim = input_dim
        
        # Add hidden layers
        for h_dim in h_dims:
            layers.extend([
                nn.Linear(current_dim, h_dim),
                nn.BatchNorm1d(h_dim),
                nn.ReLU()
            ])
            current_dim = h_dim
        
        # Output layer
        layers.append(nn.Linear(current_dim, x_dim))
        
        self.network = nn.Sequential(*layers)

    def forward(self, z, cat_covs, num_covs):
        # Process categorical covariates through embeddings
        embeddings = [emb(cat_covs[:, i]) for i, emb in enumerate(self.embeddings)]
        embedded = torch.cat(embeddings, dim=1)
        
        # Concatenate all inputs
        gen_input = torch.cat([z, embedded, num_covs], dim=1)
        
        # Generate output
        return self.network(gen_input)

In [3]:
class Discriminator(nn.Module):
    def __init__(self, x_dim, vocab_sizes, nb_numeric, h_dims):
        """
        Discriminator network for conditional GAN
        Args:
            x_dim: Dimension of input data
            vocab_sizes: List of vocabulary sizes for each categorical variable
            nb_numeric: Number of numeric covariates
            h_dims: List of hidden dimensions
        """
        super(Discriminator, self).__init__()
        
        # Embedding layers for categorical variables
        self.embeddings = nn.ModuleList([
            nn.Embedding(vocab_size, min(50, vocab_size))
            for vocab_size in vocab_sizes
        ])
        
        # Calculate total embedding dimension
        embedding_dim = sum(min(50, vocab_size) for vocab_size in vocab_sizes)
        
        # Input dimension is data dim + embedding dim + numeric covariates
        input_dim = x_dim + embedding_dim + nb_numeric
        
        # Build discriminator network
        layers = []
        current_dim = input_dim
        
        # Add hidden layers
        for h_dim in h_dims:
            layers.extend([
                nn.Linear(current_dim, h_dim),
                nn.LeakyReLU(0.2),
                nn.Dropout(0.3)
            ])
            current_dim = h_dim
        
        # Output layer
        layers.append(nn.Linear(current_dim, 1))
        
        self.network = nn.Sequential(*layers)

    def forward(self, x, cat_covs, num_covs):
        # Process categorical covariates through embeddings
        embeddings = [emb(cat_covs[:, i]) for i, emb in enumerate(self.embeddings)]
        embedded = torch.cat(embeddings, dim=1)
        
        # Concatenate all inputs
        disc_input = torch.cat([x, embedded, num_covs], dim=1)
        
        # Generate output
        return self.network(disc_input)


In [4]:
def train_gan(generator, discriminator, dataloader, cat_covs, num_covs, 
              config, device, score_fn=None, save_fn=None):
    """
    Train the conditional GAN with progress tracking and proper device handling
    """
    # Optimizers
    g_optimizer = optim.RMSprop(generator.parameters(), lr=config['lr'])
    d_optimizer = optim.RMSprop(discriminator.parameters(), lr=config['lr'])
    
    # Convert covariates to tensors and move to device
    cat_covs = torch.tensor(cat_covs, dtype=torch.long).to(device)
    num_covs = torch.tensor(num_covs, dtype=torch.float32).to(device)
    
    total_batches = len(dataloader)
    
    print(f"Starting training for {config['epochs']} epochs...")
    print(f"Total batches per epoch: {total_batches}")
    print(f"Using device: {device}")
    
    for epoch in range(config['epochs']):
        d_losses = []
        g_losses = []
        print(f"\nEpoch [{epoch+1}/{config['epochs']}]")
        
        for batch_idx, (real_data,) in enumerate(dataloader):
            batch_size = real_data.size(0)
            
            # Move real data to device
            real_data = real_data.to(device)
            
            # Get random batch of categorical and numerical covariates
            batch_indices = torch.randint(0, cat_covs.size(0), (batch_size,))
            batch_cat_covs = cat_covs[batch_indices]
            batch_num_covs = num_covs[batch_indices]
            
            # Train Discriminator
            for _ in range(config['nb_critic']):
                d_optimizer.zero_grad()
                
                # Generate fake data
                z = torch.randn(batch_size, config['latent_dim']).to(device)
                fake_data = generator(z, batch_cat_covs, batch_num_covs)
                
                # Calculate discriminator loss
                real_validity = discriminator(real_data, batch_cat_covs, batch_num_covs)
                fake_validity = discriminator(fake_data.detach(), batch_cat_covs, batch_num_covs)
                
                d_loss = -(torch.mean(real_validity) - torch.mean(fake_validity))
                d_loss.backward()
                d_optimizer.step()
                
                # Clip discriminator weights (Wasserstein GAN)
                for p in discriminator.parameters():
                    p.data.clamp_(-0.01, 0.01)
                    
                d_losses.append(d_loss.item())
            
            # Train Generator
            g_optimizer.zero_grad()
            
            # Generate fake data
            z = torch.randn(batch_size, config['latent_dim']).to(device)
            fake_data = generator(z, batch_cat_covs, batch_num_covs)
            
            # Calculate generator loss
            fake_validity = discriminator(fake_data, batch_cat_covs, batch_num_covs)
            g_loss = -torch.mean(fake_validity)
            
            g_loss.backward()
            g_optimizer.step()
            
            g_losses.append(g_loss.item())
            
            # Print progress every 10 batches
            if batch_idx % 10 == 0:
                print(f"  Batch [{batch_idx}/{total_batches}] " \
                      f"D_loss: {d_loss.item():.4f}, " \
                      f"G_loss: {g_loss.item():.4f}")
        
        # Print epoch summary
        avg_d_loss = np.mean(d_losses)
        avg_g_loss = np.mean(g_losses)
        print(f"\nEpoch {epoch+1} Summary:")
        print(f"  Average D_loss: {avg_d_loss:.4f}")
        print(f"  Average G_loss: {avg_g_loss:.4f}")
        
        # Log metrics
        if wandb.run is not None:
            wandb.log({
                'epoch': epoch,
                'd_loss': np.mean(d_losses),
                'g_loss': np.mean(g_losses)
            })
        
        # Evaluate and save model if needed
        if score_fn is not None and epoch % 10 == 0:
            score = score_fn(generator)
            print(f'Epoch {epoch}: Score = {score:.4f}')
        
        if save_fn is not None and epoch % 50 == 0:
            save_fn(generator, discriminator, epoch)

In [6]:
def main(selected_categories=None):
    """
    Train the GAN with selected categorical variables
    Args:
        selected_categories: List of column names to use as categorical variables.
                           If None, uses all columns except 'cell_id'
    """
    # Configuration
    CONFIG = {
        'epochs': 200,
        'latent_dim': 64,
        'batch_size': 32,
        'nb_layers': 2,
        'hdim': 256,
        'lr': 5e-4,
        'nb_critic': 5
    }
    
    # Device configuration
    if torch.cuda.is_available():
        device = torch.device('cuda')
    elif torch.backends.mps.is_available():
        device = torch.device('mps')
    else:
        device = torch.device('cpu')
    
    print(f"Using device: {device}")
    
    # Load data
    data_path = "/Users/guyshani/Documents/PHD/Aim_2/10x_data_mouse/"
    
    # Load expression matrix
    with h5py.File(data_path+'combined_scaled_data.h5', 'r') as f:
        x_train = f['matrix'][:].T
    
    # Load all categorical variables from single file
    cat_data = pd.read_csv(data_path+'combined_metadata.csv', sep=';')
    print("Categorical data shape:", cat_data.shape)
    print("Available categorical variables:", [col for col in cat_data.columns if col != 'cell_id'])
    
    # Determine which categories to use
    if selected_categories is None:
        # Use all columns except cell_id
        categories_to_use = [col for col in cat_data.columns if col != 'cell_id']
    else:
        # Validate selected categories
        invalid_categories = [cat for cat in selected_categories if cat not in cat_data.columns]
        if invalid_categories:
            raise ValueError(f"Invalid categories: {invalid_categories}")
        categories_to_use = selected_categories
    
    print(f"\nUsing categorical variables: {categories_to_use}")
    
    # Create dictionaries and inverse mappings for categorical variables
    cat_dicts = []
    encoded_covs = []
    
    # Process each selected column as a categorical variable
    for column in categories_to_use:
        # Get the column data
        cat_vec = cat_data[column]
        print(f"\nProcessing categorical variable: {column}")
        
        # Create list of unique category names, sorted
        dict_inv = np.array(list(sorted(set(cat_vec.values))))
        dict_map = {t: i for i, t in enumerate(dict_inv)}
        cat_dicts.append(dict_inv)
        
        # Convert categorical variables to integers
        encoded = np.vectorize(lambda t: dict_map[t])(cat_vec)
        encoded = encoded.reshape(-1, 1)  # Reshape to column vector
        encoded_covs.append(encoded)
        
        print(f"Categories in {column}:", dict_inv)
        print(f"Number of categories:", len(dict_inv))
    
    # Combine all categorical covariates
    cat_covs = np.hstack(encoded_covs)
    print("\nCombined categorical covariates shape:", cat_covs.shape)
    
    # Load numerical covariates (currently empty)
    num_covs = np.zeros((x_train.shape[0], 0))
    
    # Convert data to PyTorch tensors and move to device
    x_train = torch.tensor(x_train, dtype=torch.float32)  # Keep on CPU for DataLoader
    
    # Create data loader
    train_dataset = TensorDataset(x_train)
    train_loader = DataLoader(
        train_dataset, 
        batch_size=CONFIG['batch_size'],
        shuffle=True,
        drop_last=True
    )
    
    # Initialize models
    vocab_sizes = [len(c) for c in cat_dicts]
    print("\nVocabulary sizes for categorical variables:", vocab_sizes)
    nb_numeric = num_covs.shape[-1]
    x_dim = x_train.shape[-1]
    ##
    #print("x_train: "+str(x_train))
    #print("x_dim: "+str(x_dim))
    #print("vocab_sizes: "+str(vocab_sizes))
    
    generator = Generator(
        x_dim=x_dim,
        vocab_sizes=vocab_sizes,
        nb_numeric=nb_numeric,
        h_dims=[CONFIG['hdim']] * CONFIG['nb_layers'],
        z_dim=CONFIG['latent_dim']
    ).to(device)
    
    discriminator = Discriminator(
        x_dim=x_dim,
        vocab_sizes=vocab_sizes,
        nb_numeric=nb_numeric,
        h_dims=[CONFIG['hdim']] * CONFIG['nb_layers']
    ).to(device)
    
    # Define save function
    def save_models(generator, discriminator, epoch):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        # create save directory
        categories_str = "+".join(categories_to_use)
        save_dir = os.path.join(data_path, "saved_models")
        os.makedirs(save_dir, exist_ok=True)
        # Create run folder
        run_dir = os.path.join(save_dir, f"run_{timestamp}_{categories_str}")
        os.makedirs(run_dir, exist_ok=True)

        # Save model initialization parameters
        model_config = {
            'x_dim': x_dim,
            'vocab_sizes': vocab_sizes,
            'nb_numeric': nb_numeric,
            'h_dims': [CONFIG['hdim']] * CONFIG['nb_layers'],
            'z_dim': CONFIG['latent_dim'],
            'categories': categories_to_use,
            'training_config': CONFIG}
        config_path = os.path.join(run_dir, 'model_config.json')
        with open(config_path, 'w') as f:
            json.dump(model_config, f, indent=4)
        
        # Save generator
        generator_path = os.path.join(run_dir, f"generator_{timestamp}_{categories_str}.pt")
        torch.save(generator.state_dict(), generator_path)
        
        
        # Save discriminator
        discriminator_path = os.path.join(run_dir, f"discriminator_{timestamp}_{categories_str}.pt")
        torch.save(discriminator.state_dict(), discriminator_path)
        
        print(f"\nModels saved at epoch {epoch + 1}:")
        print(f"Generator: {generator_path}")
        print(f"Discriminator: {discriminator_path}")
        
        # Log to wandb
        if wandb.run is not None:
            wandb.save(generator_path)
            wandb.save(discriminator_path)

    # Initialize wandb with unique run name
    run_name = f"run_{int(time.time())}"  # Uses timestamp for unique name
    wandb.init(
        project='adversarial_gene_expr',
        config=CONFIG,
        name=run_name,
        reinit=True  # Ensures new run each time
    )
    
    # Add selected categories to wandb config
    wandb.config.update({'selected_categories': categories_to_use})
    
    # Train model
    train_gan(
        generator=generator,
        discriminator=discriminator,
        dataloader=train_loader,
        cat_covs=cat_covs,
        num_covs=num_covs,
        config=CONFIG,
        device=device,
        save_fn=save_models
        #save_fn=None
    )

if __name__ == '__main__':
    # Example usage:
    # Use specific categories:
    main(selected_categories=['dataset'])
    
    # Or use all available categories:
    # main()

Using device: mps
Categorical data shape: (41588, 3)
Available categorical variables: ['dataset', 'cluster']

Using categorical variables: ['dataset']

Processing categorical variable: dataset
Categories in dataset: ['dataset1' 'dataset2' 'dataset3' 'dataset4' 'dataset5' 'dataset6'
 'dataset7']
Number of categories: 7

Combined categorical covariates shape: (41588, 1)

Vocabulary sizes for categorical variables: [7]


wandb: ERROR Control-C detected -- Run data was not synced


KeyboardInterrupt: 

In [7]:
### Functions for data gneration
def inspect_generator_dims(generator):
    """
    Inspect the generator's dimensions and architecture
    
    Parameters:
        generator: Generator model
    
    Returns:
        dict containing dimension information
    """
    # Get embedding dimensions
    embedding_dims = [emb.embedding_dim for emb in generator.embeddings]
    total_embedding_dim = sum(embedding_dims)
    
    # Get first layer dimension
    first_layer_in_dim = generator.network[0].in_features
    
    return {
        'embedding_dims': embedding_dims,
        'total_embedding_dim': total_embedding_dim,
        'first_layer_in_dim': first_layer_in_dim,
        'recommended_latent_dim': first_layer_in_dim - total_embedding_dim
    }

def generate_expression_profiles(generator, n_samples, dataset_category, device='mps', debug=False):
    """
    Generate gene expression profiles using the trained cWGAN generator
    
    Parameters:
        generator: Trained Generator model
        n_samples: Number of profiles to generate
        dataset_category: Integer indicating which dataset category to generate (0-6 for dataset1-dataset7)
        device: Device to run generation on ('cuda', 'mps', or 'cpu')
        debug: If True, print debugging information
    
    Returns:
        numpy array of generated expression profiles with shape (n_samples, n_genes)
    """
    # Set generator to eval mode
    generator.eval()
    
    # Inspect dimensions
    dims = inspect_generator_dims(generator)
    
    if debug:
        print("Generator dimensions:")
        for k, v in dims.items():
            print(f"{k}: {v}")
    
    # Create latent vectors
    latent_dim = dims['recommended_latent_dim']
    z = torch.randn(n_samples, latent_dim, device=device)
    
    if debug:
        print(f"\nLatent vector shape: {z.shape}")
    
    # Create categorical condition tensor
    cat_covs = torch.full((n_samples, 1), dataset_category, dtype=torch.long, device=device)
    
    if debug:
        print(f"Categorical covariates shape: {cat_covs.shape}")
    
    # Create empty numeric covariates tensor
    num_covs = torch.zeros((n_samples, 0), device=device)
    
    # Generate samples
    try:
        with torch.no_grad():
            # Get embeddings
            embeddings = [emb(cat_covs[:, i]) for i, emb in enumerate(generator.embeddings)]
            embedded = torch.cat(embeddings, dim=1)
            
            if debug:
                print(f"Embedded shape: {embedded.shape}")
            
            # Concatenate inputs
            gen_input = torch.cat([z, embedded, num_covs], dim=1)
            
            if debug:
                print(f"Generator input shape: {gen_input.shape}")
                print(f"First layer input dim: {generator.network[0].in_features}")
                print(f"First layer weight shape: {generator.network[0].weight.shape}")
            
            # Generate samples
            fake_samples = generator.network(gen_input)
            
    except RuntimeError as e:
        print("\nError during generation:")
        print(e)
        print("\nGenerator architecture:")
        print(generator)
        raise
    
    # Convert to numpy array
    return fake_samples.cpu().numpy()

def generate_and_save_profiles(generator, n_samples_per_category, save_path, device='mps', debug=False):
    """
    Generate expression profiles for all dataset categories and save to file
    
    Parameters:
        generator: Trained Generator model
        n_samples_per_category: Number of samples to generate per dataset category
        save_path: Path to save the generated profiles
        device: Device to run generation on ('cuda', 'mps', or 'cpu')
        debug: If True, print debugging information
    """
    all_samples = []
    all_categories = []
    
    # Generate samples for each dataset category
    for category in range(7):  # 7 datasets (dataset1-dataset7)
        if debug:
            print(f"\nGenerating samples for dataset{category+1}")
        
        samples = generate_expression_profiles(
            generator, 
            n_samples_per_category, 
            category, 
            device,
            debug=debug
        )
        all_samples.append(samples)
        all_categories.extend([f'dataset{category+1}'] * n_samples_per_category)
    # Print saved data path
    print("Save location: "+str(save_path))

    # Combine all samples
    all_samples = np.vstack(all_samples)
    
    # Save generated profiles
    np.save(f'{save_path}_profiles.npy', all_samples)
    
    # Save category labels
    with open(f'{save_path}_categories.txt', 'w') as f:
        for category in all_categories:
            f.write(f'{category}\n')
            
    return all_samples, all_categories

In [ ]:
# Generate data

# Set directories
run_dir = "/Users/guyshani/Documents/PHD/Aim_2/10x_data_mouse/saved_models/run_20250112_163225_dataset/"
generator_model = "generator_20250112_163225_dataset.pt"

# Device configuration
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
    
print(f"Using device: {device}")

# Load configuration
config_path = os.path.join(run_dir, 'model_config.json')
with open(config_path, 'r') as f:
    model_config = json.load(f)
    
# Initialize models with saved configuration
generator = Generator(
    x_dim=model_config['x_dim'],
    vocab_sizes=model_config['vocab_sizes'],
    nb_numeric=model_config['nb_numeric'],
    h_dims=model_config['h_dims'],
    z_dim=model_config['z_dim']).to(device)
    
discriminator = Discriminator(
    x_dim=model_config['x_dim'],
    vocab_sizes=model_config['vocab_sizes'],
    nb_numeric=model_config['nb_numeric'],
    h_dims=model_config['h_dims']).to(device)



#discriminator_path = os.path.join(run_dir, "discriminator.pt")
#discriminator.load_state_dict(torch.load(discriminator_path, map_location=device, weights_only=True))
generator_path = os.path.join(run_dir, generator_model)
generator.load_state_dict(torch.load(generator_path, map_location=device, weights_only=True))


all_samples, categories = generate_and_save_profiles(
    generator,
    n_samples_per_category=100,
    save_path=run_dir+'generated_data',
    debug=True  # Enable debug output
)

Using device: mps

Generating samples for dataset1
Generator dimensions:
embedding_dims: [7]
total_embedding_dim: 7
first_layer_in_dim: 71
recommended_latent_dim: 64

Latent vector shape: torch.Size([100, 64])
Categorical covariates shape: torch.Size([100, 1])
Embedded shape: torch.Size([100, 7])
Generator input shape: torch.Size([100, 71])
First layer input dim: 71
First layer weight shape: torch.Size([256, 71])


/var/folders/g_/c_f20pc566n31j9p36lcdc0w0000gn/T/ipykernel_19458/2294553458.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  generator.load_state_dict(torch.load(generat


Generating samples for dataset2
Generator dimensions:
embedding_dims: [7]
total_embedding_dim: 7
first_layer_in_dim: 71
recommended_latent_dim: 64

Latent vector shape: torch.Size([100, 64])
Categorical covariates shape: torch.Size([100, 1])
Embedded shape: torch.Size([100, 7])
Generator input shape: torch.Size([100, 71])
First layer input dim: 71
First layer weight shape: torch.Size([256, 71])

Generating samples for dataset3
Generator dimensions:
embedding_dims: [7]
total_embedding_dim: 7
first_layer_in_dim: 71
recommended_latent_dim: 64

Latent vector shape: torch.Size([100, 64])
Categorical covariates shape: torch.Size([100, 1])
Embedded shape: torch.Size([100, 7])
Generator input shape: torch.Size([100, 71])
First layer input dim: 71
First layer weight shape: torch.Size([256, 71])

Generating samples for dataset4
Generator dimensions:
embedding_dims: [7]
total_embedding_dim: 7
first_layer_in_dim: 71
recommended_latent_dim: 64

Latent vector shape: torch.Size([100, 64])
Categorica

In [32]:
## Load generated data
# Load the generated profiles
profiles = np.load(run_dir + 'generated_data_profiles.npy')

# Load categories
with open(run_dir + 'generated_data_categories.txt', 'r') as f:
    categories = [line.strip() for line in f]

# Convert to pandas DataFrame
df = pd.DataFrame(profiles)

# Add categories as a column
df['dataset'] = categories
df

,0,1,2,3,4,5,6,7,8,9,...,41579,41580,41581,41582,41583,41584,41585,41586,41587,dataset
0,0.836664,-0.173137,-0.048852,-0.056615,-0.111151,-0.213191,0.098130,0.008074,-0.082588,-0.193292,...,-0.233236,0.857102,0.323777,0.100324,-0.346655,-0.270658,0.257891,-0.242051,-0.060632,dataset1
1,0.420519,-0.878203,-0.016036,-0.649573,-0.898842,-0.709133,-0.478758,-0.900632,-0.554757,-0.784076,...,-0.859491,-1.149823,3.508285,0.040627,-1.053828,-1.396616,4.058584,-1.391942,-1.613365,dataset1
2,0.257364,-0.410113,-0.168651,0.019554,0.605600,0.025765,0.105883,0.720470,0.320675,-0.161310,...,-0.339561,0.272903,0.032933,-0.193340,-0.125451,0.147760,-0.238648,-0.175982,0.486519,dataset1
3,-0.041994,-0.096628,0.107887,0.093147,-0.119546,-0.143102,0.205479,-0.036449,-0.087694,-0.096305,...,-0.187571,-0.296984,0.569445,0.141704,-0.155149,-0.222770,0.511674,-0.196746,-0.231371,dataset1
4,0.084030,0.034210,0.217438,0.271770,-0.120320,-0.120483,0.535898,0.032819,-0.073668,0.045240,...,-0.050118,-0.015386,0.251784,0.312822,0.039282,-0.125010,-0.082538,-0.085942,-0.000719,dataset1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,0.553130,-0.289484,-0.126051,-0.236413,-0.109200,-0.313937,-0.086129,-0.024629,-0.228695,-0.270952,...,-0.474593,0.234379,0.966087,-0.131921,-0.516797,-0.442748,0.957596,-0.505236,-0.189231,dataset7
696,1.734083,-0.456024,-0.048328,0.227071,0.323782,-0.609462,0.458538,0.750408,0.031424,-0.096684,...,-0.763727,1.545573,0.839734,0.435531,-0.558352,-0.387740,0.165579,-0.834695,0.145693,dataset7
697,1.221476,0.087605,0.694797,0.768075,-0.355958,-0.615676,1.176754,0.196508,-0.049763,0.150603,...,0.100679,1.442197,0.389280,1.605862,-0.531913,-0.458261,-0.405126,-0.386913,0.078110,dataset7
698,0.976643,-0.489326,0.027495,-0.162150,0.252294,0.087300,-0.259395,0.436169,0.108348,-0.316288,...,-0.410118,1.267244,0.025311,0.254876,0.016877,0.170417,0.096193,-0.365324,0.285262,dataset7
